In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import sys
sys.path.append('/opt/conda/envs/python37/lib/python3.7/site-packages')
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point, Polygon
from geopandas import GeoSeries,GeoDataFrame
import folium.plugins as plugins
import re
import seaborn as sns
import mapclassify
import PIL
import io
import os
import plot_map
from shapely.ops import nearest_points
import math
import statsmodels.api as sm

### data processing 

In [ ]:
# 加入地图的信息
grid = gpd.read_file('/root/Data/泉州整合空间数据【0426】/泉州整合空间数据【0426】/yuwang/qzyw.shp')
grid = grid.to_crs("EPSG:4326")
grid['center'] = grid['geometry'].representative_point()
grid['lng'] = grid['center'].x
grid['lat'] = grid['center'].y
grid.rename(columns={'FID_1':'FID'},inplace=True)
IDID_list = grid['IDID'].tolist()


In [ ]:
grid

In [ ]:
## 行政区划

attr = pd.read_excel('/root/Data/泉州整合空间数据【0426】/泉州整合空间数据【0426】/QZ_指标finally【0426】.xlsx')
district = set(attr['行政区'].to_list())
print(district)

quhua = gpd.read_file('/root/Data/quhua/ok_geo.csv')
qz_quhua = quhua.iloc[1171:1184,:]

def geo(polygon):
    t = re.split('[,;]',polygon)
    t = [i.split(' ') for i in t]
    t = [[float(i[0]),float(i[1])] for i in t]
    return Polygon(t)

qz_quhua['geometry'] = qz_quhua.apply(lambda x: geo(x['polygon']), axis=1)

dis = list(district)[2:]
dis = pd.DataFrame(dis,columns=['name'])
dis_shp = qz_quhua.merge(dis,how='right')
dis_shp

tmp_c = quhua.iloc[1171]['geo'].split(' ')
city_map = folium.Map(location=[float(tmp_c[1]),float(tmp_c[0])],zoom_start=10)
folium.GeoJson(
    qz_quhua[1:].iloc[1:].to_json(),
).add_to(city_map)
city_map

qu_shp = gpd.read_file('/root/Projects/dengwanting/grid_quhua/quhua.shp')


In [ ]:
data_per_half_hour_all = pd.read_csv('/root/zp/processedData/per_half_hour/slice_data/qz_lbs_20200218_slice.csv')

data_per_half_hour_all_ = gpd.GeoDataFrame(
    data_per_half_hour_all.loc[:, [c for c in data_per_half_hour_all.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(data_per_half_hour_all["geometry"]),
    crs="epsg:4326",
)

data_per_half_hour_all_gpd = gpd.GeoDataFrame(data_per_half_hour_all_,geometry='geometry')
fig, ax = plt.subplots(figsize=(10, 10))
data_per_half_hour_all_gpd.plot(ax=ax,cmap='Reds',scheme='userdefined',classification_kwds={'bins':[0,10,50,100,200,550]},column='H0',legend=True)
qu_shp.plot(ax=ax,edgecolor = (0,0,0,1),facecolor = (0,0,0,0),linewidths = 0.4)
plt.show()

In [ ]:
data_selected = data_per_half_hour_all_gpd.copy()

In [ ]:
## 地块编号矩阵
min_x,max_x,min_y,max_y = 118.4159,118.9855,24.6397,25.2051
interval_x,interval_y = 0.00494,0.00451
# row,col = math.ceil((max_y-min_y)/interval_y),math.ceil((max_x-min_x)/interval_x)
# row,col
# index_mat = np.zeros([row,col])

# for i in range(len(data_selected)):
#     x = math.floor((data_selected.iloc[i]['lng']-min_x)/interval_x + 0.5)
#     y = math.floor((data_selected.iloc[i]['lat']-min_y)/interval_y + 0.5)
#     index_mat[y][x] = data_selected.iloc[i]['IDID']
index_mat = np.load('/root/zp/flow_data/index_mat.npy')
row,col = index_mat.shape

In [ ]:
## 人流密度矩阵
density_mat = dict()
for i in range(48):
    key = 'density_mat_H' + str(i)
    # value = np.zeros([row,col])
    # time = 'H' + str(i)
    # for j in range(len(data_selected)):
    #     x = math.floor((data_selected.iloc[j]['lng']-min_x)/interval_x + 0.5)
    #     y = math.floor((data_selected.iloc[j]['lat']-min_y)/interval_y + 0.5)
    #     value[y][x] = data_selected.iloc[j][time]
    # np.save('/root/zp/flow_data/20190219/density_mat/' + key + '.npy', value)
    # density_mat[key] = value
    density_mat[key] = np.load('/root/zp/flow_data/20190219/density_mat/' + key + '.npy')

In [ ]:
## 计算源与流
for i in range(48):
    key = 'density_mat_H' + str(i)
    
    test_density_mat = density_mat[key]
    test_density_mat_pad = np.pad(test_density_mat, 1)  # 矩阵外圈补0

    test_divergence_mat = np.zeros([row, col])
    test_gradient_x_mat = np.zeros([row, col])
    test_gradient_y_mat = np.zeros([row, col])

    for j in range(1, test_density_mat_pad.shape[0]-1):
        for k in range(1, test_density_mat_pad.shape[1]-1):
            # -4 1
            test_divergence_mat[j-1, k-1] = (test_density_mat_pad[j-1, k] + test_density_mat_pad[j+1, k] + \
                                             test_density_mat_pad[j, k-1] + test_density_mat_pad[j, k+1]) - (4 * test_density_mat_pad[j, k])
            # -8 1
            # test_divergence_mat[j-1, k-1] = (test_density_mat_pad[j-1, k] + test_density_mat_pad[j+1, k] + \
            #                         test_density_mat_pad[j, k-1] + test_density_mat_pad[j, k+1] + test_density_mat_pad[j-1,k-1] + \
            #                         test_density_mat_pad[j-1,k+1] + test_density_mat_pad[j+1, k-1] + test_density_mat_pad[j+1, k+1]) \
            #                         - (4 * test_density_mat_pad[j, k])
            
            # test_gradient_x_mat[j-1, k-1] = test_density_mat_pad[j, k+1] - test_density_mat_pad[j, k-1]
            # test_gradient_y_mat[j-1, k-1] = test_density_mat_pad[j-1, k] - test_density_mat_pad[j+1, k]
            
    np.save('/root/zp/flow_data/20190219/divergence_mat/' + 'divergence_mat_H' + str(i) + '.npy', test_divergence_mat)
    # np.save('/root/zp/flow_data/20190219/gradient_x_mat/' + 'gradient_x_mat_H' + str(i) + '.npy', test_gradient_x_mat)
    # np.save('/root/zp/flow_data/20190219/gradient_y_mat/' + 'gradient_y_mat_H' + str(i) + '.npy', test_gradient_y_mat)
    
    # test_divergence_mat_adjust = np.zeros([row, col])
    # for m in range(row):
    #     for n in range(col):
    #         if test_divergence_mat[m, n] > 0:
    #             test_divergence_mat_adjust[m, n] = 1
    #         elif test_divergence_mat[m, n] < 0:
    #             test_divergence_mat_adjust[m, n] = -1
                
    # key2 = 'divergence_H' + str(i)
    # data_per_half_hour_all_gpd[key2] = 0

    # for u in range(row):
    #     for v in range(col):
    #         if index_mat[u, v] != 0:
    #             block_id = index_mat[u, v]
    #             index = data_per_half_hour_all_gpd[data_per_half_hour_all_gpd.IDID == block_id].index.values[0]
    #             # data_per_half_hour_all_gpd.loc[index, key2] = test_divergence_mat_adjust[u, v]
    #             data_per_half_hour_all_gpd.loc[index, key2] = test_divergence_mat[u, v]

In [ ]:
a = data_per_half_hour_all_gpd.copy()

b = ['IDID','lng','lat','geometry']
for i in range(48):
    a.rename(columns={'divergence_H' + str(i) : 'div_H' + str(i)}, inplace=True)
    b.append('div_H' + str(i))
a = a[b]
a.to_file('/root/zp/flow_data/20190219/divergence_new.shp')

### drawing